In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as la
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.base import clone
import timeit
import math
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings('ignore')

In [2]:
def pca(F, X):
    n, d = X.shape
    mu = np.zeros((d, 1))
    Z = np.zeros((d, F))
    for i in range(d):
        mu[i] = (1. / n) * np.sum(X[:, [i]])
    X = X - mu.T
    U, s, Vt = la.svd(X, False)
    g = s[:F]
    for i in range(F):
        g[i] = 1. / g[i]
    W = Vt[:F]
    Z = np.dot(W.T, np.diag(g))
    return (mu, Z)

def pca_proj(X,mu,Z):
    n, d = X.shape
    X = X - mu.T
    return np.dot(X, Z)

In [3]:
def k_fold(k, model, X, y):
    n, d = X.shape
    z = np.zeros((k, 1))
    for i in range(k):
        T = list(range(int((i * n) / k), int((n * (i + 1) / k))))
        S = [j for j in range(n) if j not in T]
        curr_model = clone(model)
        curr_model.fit(X[S], y[S])
        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(X[T])) ** 2)
    return z

In [4]:
def bootstrapping(B, model, X, y):
    n, d = X.shape
    z = np.zeros((B, 1))
    for i in range(B):
        u = np.random.choice(n, n, replace=True)
        S = np.unique(u)
        T = np.setdiff1d(np.arange(n), S, assume_unique=True)
        curr_model = clone(model)
        curr_model.fit(X[u], y[u])
        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        # theta_hat will be d by 1
        z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(X[T])) ** 2)
    return z

In [31]:
def evaluate_model(model, f, X, y, k=5, B=5):
    ######################## KFOLD ###################
    print('Evaluating K-fold with %d folds.' % k)
    start_time = timeit.default_timer()
    k_fold_z = k_fold(k, model, f, X, y, error_type="log_mse")
    elapsed = timeit.default_timer() - start_time
    
    k_fold_mse = np.mean(k_fold_z)
    print('K-fold Mean Squared log Error: ', k_fold_mse)
    
    k_fold_rmse = math.sqrt(k_fold_mse)
    print('K-fold Square Root Mean Squared log Error: ', k_fold_rmse)

    print("Time elapsed for k-fold: ", elapsed)

    print()
    print()
    
    ################### BOOTSTRAPPING ################
    print('Evaluating bootstrapping with %d bootstraps.' % B)
    start_time = timeit.default_timer()
    bootstrapping_z = bootstrapping(B, model, f, X, y)
    elapsed = timeit.default_timer() - start_time

    bootstrapping_mse = np.mean(bootstrapping_z)
    print('Bootstrapping Mean Squared Error: ', bootstrapping_mse)

    bootstrapping_rmse = math.sqrt(bootstrapping_mse)
    print("Time elapsed for bootstrapping: ", elapsed)
    
    return (k_fold_z, k_fold_mse, k_fold_rmse, bootstrapping_z, bootstrapping_mse, bootstrapping_rmse)

# Data Processing

In [6]:
data = pd.read_csv("train.csv", header=0)
print(data.shape)

X = data.iloc[:,:-1]
Y = data.iloc[:,-1:]

print(X.shape)
print(Y.shape)

(1460, 81)
(1460, 80)
(1460, 1)


In [7]:
# this just sums up how many nulls per feature and divides to find percentage of nulls per feature
# if over 50% null then print the feature
data_keys = X.keys()
for i, b in enumerate((X.isnull().sum() / X.shape[0]) > 0.5):
    if b:
        print(data_keys[i])

Alley
PoolQC
Fence
MiscFeature


In [8]:
# data = data.drop(['Alley', 'MiscFeature', 'Fence', 'PoolQC'], axis=1)

In [9]:
# Replaces categorical value in Quality columns with numerical scale
qualityCols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC',
              'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond']

data[qualityCols].head()

for col in qualityCols:
    # NA is never used since all NA's got converted to NaN objects when pandas read in the csv
    data[col] = data[col].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1, 'NA': 0})

data[qualityCols].head()

,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond
0,4,3,4.0,3.0,5,4,NaN,3.0,3.0
1,3,3,4.0,3.0,5,3,3.0,3.0,3.0
2,4,3,4.0,3.0,5,4,3.0,3.0,3.0
3,3,3,3.0,4.0,4,4,4.0,3.0,3.0
4,4,3,4.0,3.0,5,4,3.0,3.0,3.0


In [10]:
# categorical columns
catCols = set(list(X))-set(list(X._get_numeric_data()))
print(catCols)

#Fill Categorical Column Null values with 0
for col in catCols:
    data[col].fillna(0, inplace=True)

#Fill numerical column null values with mean of column
data = data.fillna(data.mean())

{'BsmtExposure', 'GarageCond', 'MSZoning', 'FireplaceQu', 'BsmtFinType2', 'PavedDrive', 'Functional', 'BsmtCond', 'LandSlope', 'SaleType', 'KitchenQual', 'GarageType', 'ExterQual', 'Utilities', 'LandContour', 'GarageFinish', 'Exterior1st', 'Electrical', 'Exterior2nd', 'HouseStyle', 'LotConfig', 'BsmtQual', 'PoolQC', 'SaleCondition', 'Alley', 'Heating', 'Condition1', 'MiscFeature', 'Foundation', 'Condition2', 'Neighborhood', 'HeatingQC', 'MasVnrType', 'RoofMatl', 'LotShape', 'BldgType', 'ExterCond', 'BsmtFinType1', 'CentralAir', 'RoofStyle', 'Street', 'GarageQual', 'Fence'}


In [11]:
#Perform one hot encoding on all categorical columns
frames = []
for col in catCols:
    oneHot_encoded = pd.get_dummies(X[col])
    oneHot_encoded = oneHot_encoded.add_prefix(col + '_is_')
    frames.append(oneHot_encoded)

X = X.drop(catCols, axis=1)

X = pd.concat(frames, axis=1)

In [12]:
X.keys()

Index(['BsmtExposure_is_Av', 'BsmtExposure_is_Gd', 'BsmtExposure_is_Mn',
       'BsmtExposure_is_No', 'GarageCond_is_Ex', 'GarageCond_is_Fa',
       'GarageCond_is_Gd', 'GarageCond_is_Po', 'GarageCond_is_TA',
       'MSZoning_is_C (all)',
       ...
       'Street_is_Pave', 'GarageQual_is_Ex', 'GarageQual_is_Fa',
       'GarageQual_is_Gd', 'GarageQual_is_Po', 'GarageQual_is_TA',
       'Fence_is_GdPrv', 'Fence_is_GdWo', 'Fence_is_MnPrv', 'Fence_is_MnWw'],
      dtype='object', length=252)

In [13]:
X.isnull().values.any()

False

In [14]:
# 80:20 train test ratio
test_size = 0.2
# This function splits the training and target sets into random train and test subsets.
# X_train and X_test are subsets of the training data
# y_train and y_test are subsets the the target data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)

# PCA Feature Selection

In [15]:
F = 50

In [16]:
X_mu, X_Z = pca(F, X.values)
X_pca = pca_proj(X.values, X_mu, X_Z)

In [17]:
print(X_mu.shape)
print(X_Z.shape)
print(X_pca.shape)

(252, 1)
(252, 50)
(1460, 50)


In [18]:
X_train_mu, X_train_Z = pca(F, X_train.values)

In [19]:
print(X_train_mu.shape)
print(X_train_Z.shape)

(252, 1)
(252, 50)


In [20]:
X_train_pca = pca_proj(X_train.values, X_train_mu, X_train_Z)
print(X_train_pca.shape)

X_test_pca = pca_proj(X_test.values, X_train_mu, X_train_Z)
print(X_test_pca.shape)

(1168, 50)
(292, 50)


# Neural Network

In [21]:
from sklearn.neural_network import MLPRegressor
import copy

def nnRMSE(nn, X, y, X_test, y_test):
    train_pred = nn.predict(X)
    test_pred = nn.predict(X_test)
    
    trainSE = 0
    for i in range(len(train_pred)):
        trainSE += (train_pred[i]-y[i])**2
    
    testSE = 0
    for i in range(len(test_pred)):
        testSE += (test_pred[i]-y_test[i])**2
    
    trainRMSE = np.sqrt(trainSE / len(train_pred))
    testRMSE = np.sqrt(testSE / len(test_pred))
    
    return trainRMSE, testRMSE

# Based on Early Stopping
def nn_setBestWeights(nn, cvErrors, models):
    model_idx = np.argmin(cvErrors)
    nn.coefs_ = models[model_idx][0]
    nn.intercepts_ = models[model_idx][1]
    print("Loaded nn with weights of lowest model.")

# 80/20 training / cross_val split
train_size = int(len(X_pca) * .8)
nn_X_train_pca = X_pca[:train_size]
nn_X_test_pca = X_pca[train_size:]
nn_y_train_pca = Y.values.ravel()[:train_size]
nn_y_test_pca = Y.values.ravel()[train_size:]


In [40]:
# Early Stopping Routine
lr = 0.02 
num_iters = 10
nn = MLPRegressor(
                    hidden_layer_sizes=(24,24,24,),
                    activation='relu',
                    solver='adam',
                    learning_rate='adaptive',
                    warm_start=True,
                    max_iter=1,
                    learning_rate_init=0.01,
                    alpha=0.01)

max_iters = 1000
cvErrors = []
models = []
for i in range(max_iters):
    nn.fit(nn_X_train_pca, nn_y_train_pca)
    if i % 10 == 0:
        error = nnRMSE(nn, nn_X_train_pca, nn_y_train_pca, nn_X_test_pca, nn_y_test_pca)
        if i % 100 == 0 :
            print(f"(Train, test): {error}")
        models.append((copy.deepcopy(nn.coefs_), copy.deepcopy(nn.intercepts_)))
        cvErrors.append(error[1])

(Train, test): (197025.5049622786, 199795.308294238)
(Train, test): (52765.29572352412, 59715.32939736021)
(Train, test): (39494.99968096905, 47930.32504759745)
(Train, test): (39500.69437625668, 47608.204845985885)
(Train, test): (39499.86080842582, 47496.89765291244)
(Train, test): (39597.12948268359, 47696.686193593065)
(Train, test): (39522.53833684413, 47404.029453365896)
(Train, test): (39494.00674589041, 47824.53416350405)
(Train, test): (39524.120430652554, 47894.29700136297)
(Train, test): (39510.72950936185, 47638.574929508584)


In [42]:
nn_setBestWeights(nn, cvErrors, models)
nnRMSE(nn, nn_X_train_pca, nn_y_train_pca, nn_X_test_pca, nn_y_test_pca)
evaluate_model(nn, f, X_pca, Y.values.ravel(), k=5, B=0)

Loaded nn with weights of lowest model.


NameError: name 'f' is not defined

# Hyperparameter Tuning

In [ ]:
#This takes time
def tune_hyperparameters():
    adaboost_param_tuning = pd.DataFrame(columns=['parameter', 'rmse'])
    xgb_param_tuning = pd.DataFrame(columns=['parameter', 'rmse'])    
    svr_param_tuning = pd.DataFrame(columns=['parameter', 'rmse'])    

    #Tuning n estimators parameter for boosting algorithms
    for i in range(25,200,25):
        print("Boosting: " + str(i))
        adaBoost = AdaBoostRegressor(n_estimators=i)
        k_fold_rmse, k_fold_z, bootstrapping_z = evaluateModel(adaBoost, X_pca, Y.values.ravel(), k=5, B=5, verbose=False)
        adaboost_param_tuning = adaboost_param_tuning.append({'parameter': i, 'rmse': k_fold_rmse}, ignore_index=True)
        xgb = XGBRegressor(max_depth=3, learning_rate=0.2, booster='gbtree', n_estimators=i)
        k_fold_rmse, k_fold_z, bootstrapping_z = evaluateModel(xgb, X_pca, Y.values.ravel(), k=5, B=5, verbose=False)
        xgb_param_tuning = xgb_param_tuning.append({'parameter': i, 'rmse': k_fold_rmse}, ignore_index=True)
        
    #for i in range(25,200,25):
    c_vals = [0.01, 0.1, 10, 100]
    for i in c_vals:
        print("C: " + str(i))
        svr_model = svm.SVR(kernel="poly", coef0=-3500, gamma="auto", C=i)
        evaluateModel(svr_model, X_pca, Y.values.ravel(), k=5, B=5, verbose=False)
        svr_param_tuning = svr_param_tuning.append({'parameter': i, 'rmse': k_fold_rmse}, ignore_index=True)

    return xgb_param_tuning, adaboost_param_tuning, svr_param_tuning
    

In [ ]:
xgb_params, adaboost_params, svm_params = tune_hyperparameters()

# Plots

## Learning Curves

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    plt.show()


In [ ]:
plot_learning_curve(estimator=adaBoost, title="Learning Curves (AdaBoost)", X=X_train_pca, y=y_train, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))
plot_learning_curve(estimator=xgb, title="Learning Curves (XgBoost)", X=X_train_pca, y=y_train, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))
plot_learning_curve(estimator=svr_model, title="Learning Curves (SVR)", X=X_train_pca, y=y_train, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5))

## Scatter Plots

In [ ]:
def plotScatter(predicted, name):
    colors = ["r", "b"]
    plt.title(name + " Predicted vs Actual Sale Price")
    plt.xlabel("Actual Sale Price")
    plt.ylabel("Predicted Sale Price")
    red_patch = mpatches.Patch(color='red', label='Actual Sale Price')
    blue_patch = mpatches.Patch(color='blue', label='Predicted Sale Price')
    plt.legend(handles=[red_patch, blue_patch])
    plt.scatter(predicted['SalePrice'], predicted['predicted'], color=colors, alpha=0.5)
    plt.show()

In [ ]:
plotScatter(ada_pred, "AdaBoost")
plotScatter(xgb_pred, "XgBoost")
plotScatter(svr_pred, "SVM")

## Parameter Tuning

In [ ]:
def plot_param_tuning(xgb_params, adaboost_params, svm_params):
    plt.plot(adaboost_params['parameter'], adaboost_params['rmse'], marker='o', color='b')
    plt.title("Adaboost RMSE vs n_estimators")
    plt.xlabel("n_estimators")
    plt.ylabel("RMSE")
    plt.show()
    
    plt.plot(xgb_params['parameter'], xgb_params['rmse'], marker='o', color='b')
    plt.title("XgBoost RMSE vs n_estimators")
    plt.xlabel("n_estimators")
    plt.ylabel("RMSE")
    plt.show()
    
    plt.plot(svm_params['parameter'], svm_params['rmse'], marker='o', color='b')
    plt.title("SVM RMSE vs C")
    plt.xlabel("C")
    plt.ylabel("RMSE")
    plt.show()

In [ ]:
plot_param_tuning(xgb_params, adaboost_params, svm_params)